# Setting Up a PostgreSQL Database for Sterling Vet Services

You are a freelance developer who has accepted a job to develop a database for a veterinarian company.  They have given you a small CSV file and asked you to populate the database with a single table holding the data in the file. There are notes attached asking for you to add a weight for Petra of 12.5lbs  and to add a new pet Esmerelda who is a 2.5 yr old female Angus cow that weighs 1250 lbs, has no health issues, is vaccinated and is owned by the Garcia Ranch. 

### Setup PostgreSQL for cloud_user Access

Create a cloud_user database and a cloud_user user with a password.  Grant all priveleges to database cloud_user by user cloud_user.

#### Start psql
- `sudo -u postgres psql`

#### Create Database
- `CREATE DATABASE cloud_user;`

#### Create User
- `CREATE USER cloud_user WITH ENCRYPTED PASSWORD 'cloud_user';`

#### Grant Access to Database by User
- `GRANT ALL PRIVILEGES ON DATABASE cloud_user TO cloud_user;`

#### Leave psql
- `\q`

### Imports and database connection string.  The PostgreSQL standard port is 5432.

In [ ]:
import pandas as pd
import psycopg2

CONNECT_DB = "host=localhost port=5432 dbname=cloud_user user=cloud_user password=cloud_user"

### Create Table

Create a table with columns matching the tips.csv file.

In [ ]:
create_table_query = '''CREATE TABLE customers (
    id SERIAL PRIMARY KEY,
    name varchar (25),
    owner varchar (25),
    type varchar (25),
    breed varchar (25),
    color varchar (25),
    age smallint,
    weight float4,
    gender varchar (1),
    health_issues boolean,
    indoor_outdoor varchar(10),
    vaccinated boolean
); '''

try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)

    # Create a cursor to db
    cur = cxn.cursor()
    
    # Send sql query to request
    cur.execute(create_table_query)
    records = cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")

    print(f'Records: {records}')

### Add the Data to Table

Use a try..except..finally block to load the data from vet.csv into the table just created.

In [ ]:
try:
    # Make connection to db
    cxn = psycopg2.connect(CONNECT_DB)
    
    # Create a cursor to db
    cur = cxn.cursor()
    
    # read file, copy to db
    with open('./vet.csv', 'r') as f:
        # skip first row, header row
        next(f)
        cur.copy_from(f, 'customers', sep=",")
        cxn.commit()

except (Exception, psycopg2.Error) as error :
    print ("Error while connecting to PostgreSQL", error)
    
finally:
    #closing database connection.
    if(cxn):
        cur.close()
        cxn.close()
        print("PostgreSQL connection is closed")
        print("customers table populated")


### Selecting Data From a Server

Create a function to execute a SQL statement to fetch records from the database.  Use try..except..finally and .fetchall().  The user should use LIMIT or TOP() to limit their results.


In [ ]:
def db_server_fetch(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur = cxn.cursor()

        # Send sql query to request
        cur.execute(sql_query)
        records = cur.fetchall()

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")
        return records

Get all data from the database.

In [ ]:
select_query = '''SELECT * FROM customers;'''

records = db_server_fetch(select_query)
print(records)


### Change Data in Database

Create a function to execute a SQL statement to update records in the database.  Use try..except..finally. 


In [ ]:
def db_server_change(sql_query):
    try:
        # Make connection to db
        cxn = psycopg2.connect(CONNECT_DB)

        # Create a cursor to db
        cur = cxn.cursor()

        # Send sql query to request
        cur.execute(sql_query)
        records = cxn.commit()

    except (Exception, psycopg2.Error) as error :
        print ("Error while connecting to PostgreSQL", error)

    finally:
        #closing database connection.
        if(cxn):
            cur.close()
            cxn.close()
            print("PostgreSQL connection is closed")
        return records

Add a new record with the following data:
Esmerelda is a 2.5 yr old female Angus cow that weighs 1250 lbs, has no health issues, and is vaccinated and owned by the Garcia Ranch.

In [ ]:
add_data = '''INSERT INTO customers
    (name, owner, type, breed, color, age, weight, gender, health_issues, indoor_outdoor, vaccinated)
    VALUES
    ('Esmerelda', 'Garcia Ranch', 'Cattle', 'Angus', 'black', 2.5, 1250, 'f', false, 'outdoor', true);'''

db_server_change(add_data)


Check Record Added

In [ ]:
select_query = '''SELECT * FROM customers WHERE name = 'Esmerelda';'''

records = db_server_fetch(select_query)
print(records)

#### Update Petra's weight.

Make Petra's weight 12.5.

In [ ]:
update_data = '''UPDATE customers SET weight = 12.5 WHERE name = 'Petra';'''

db_server_change(update_data)

Check Record

In [ ]:
select_query = '''SELECT * FROM customer WHERE name = 'Petra';'''

records = db_server_fetch(select_query)
print(records)